# MNIST from Scratch

Can I train a model to recognize handwritten digits using numpy?

1. Load the MNIST dataset from the web and store as NumPy arrays
2. Train a simple model to solve MNIST using PyTorch
3. Do the same with NumPy by implementing various ML algorithms

In [1]:
from pathlib import Path

import numpy as np
import pandas as pd
from tqdm import trange
from torch import nn, Tensor, optim

Load data from CSV files

In [2]:
def load_dataset(data_dir: Path, file_name: str) -> np.ndarray:
    # can be done with numpy.genfromtxt(), but is much slower
    dataset = pd.read_csv(data_dir / file_name)
    X = dataset.drop("label", axis=1).to_numpy() / 255.0
    y = dataset["label"].to_numpy()
    return X, y

In [3]:
mnist_path = Path(".").resolve(strict=True).parent / "data" / "mnist"
X_train, y_train = load_dataset(mnist_path, "mnist_train.csv")
X_test, y_test = load_dataset(mnist_path, "mnist_test.csv")

Solve with PyTorch

In [4]:
class NeuralNet(nn.Module):

    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(784, 256, bias=False)
        self.layer2 = nn.Linear(256, 64, bias=False)
        self.layer3 = nn.Linear(64, 10, bias=False)
        self.act = nn.ReLU()
    
    def forward(self, x: Tensor) -> Tensor:
        x = x.view(-1, 28 * 28)
        x = self.act(self.layer1(x))
        x = self.act(self.layer2(x))
        x = self.layer3(x)
        return x

In [5]:
iterations = 3000
batch_size = 64
learning_rate = 0.001

model = NeuralNet()
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [6]:
# training loop
for i in (t := trange(iterations)):
    sample = np.random.randint(0, len(X_train), size=batch_size)
    out = model(Tensor(X_train[sample]))
    loss = loss_func(out, Tensor(y_train[sample]).long())
    loss.backward()
    optimizer.step()
    model.zero_grad()
    t.set_description(f"Iteration {i} loss {loss.item():.2f}")

Iteration 2999 loss 0.21: 100%|██████████| 3000/3000 [00:17<00:00, 169.57it/s]


In [7]:
# test accuracy
pred = model(Tensor(X_test)).argmax(dim=1)
accuracy = (pred == Tensor(y_test)).float().mean().item()
accuracy

0.9769999980926514

Now with NumPy

In [12]:
w1 = model.layer1.weight.detach().numpy()
w2 = model.layer2.weight.detach().numpy()
w3 = model.layer3.weight.detach().numpy()

In [16]:
def relu(x: np.ndarray) -> np.ndarray:
    x = np.maximum(x, 0)
    return x

In [38]:
def forward(x: np.ndarray) -> np.ndarray:
    print(x.shape, w1.shape, w2.shape)
    x = relu(x.dot(w1.T))
    x = relu(x.dot(w2.T))
    x = x.dot(w3.T)
    return x

In [39]:
pred = forward(X_test).argmax(axis=1)
accuracy = (pred == y_test)
accuracy

(10000, 784) (256, 784) (64, 256)


ValueError: shapes (10,64) and (10000,64) not aligned: 64 (dim 1) != 10000 (dim 0)